# **Personalized Product Recommendations with ML PySpark and OpenAI Embeddings**

> This Project is designed to:  
- **build scalable recommendation systems** to enhance customer experiences and drive sales.  
- Construct a **data processing pipeline** using **PySpark**.  
- Implement **K-means clustering** with **OpenAI text embeddings**.  
- Develop a **recommendation system** that suggests products based on user behavior.  



### Task 1 - Set up the project

Installing the needed modules.

In [1]:
!pip install openai==1.16.2 python-dotenv pyspark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


Imporint the modules

In [42]:
from dotenv import load_dotenv
import os
import openai
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, FloatType

from pyspark.ml.feature import VectorAssembler, PCA
from pyspark.ml.clustering import KMeans
import plotly.express as px

Setup the OpenAI API

In [43]:
from dotenv import load_dotenv

# Loading API key and organization ID from dotenv file
load_dotenv(dotenv_path='apikey.env.txt')

# Retrieving API Key from the env variables
APIKEY = os.getenv("APIKEY")

# Check if the APIKEY is loaded correctly
if APIKEY is None:
  raise ValueError("API key not found.")

openai.api_key = APIKEY

Create a Spark session

In [30]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ProductRecommenderSystem").getOrCreate()

spark

Loading the dataset

In [31]:
file_path = "products_dataset.csv"

df = spark.read.csv(file_path, header=True, inferSchema=True)

df.show()

+----------+--------------------+--------------------+
|product_id|               title|         description|
+----------+--------------------+--------------------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|
|        P1|Turmode 30 ft. RP...|If you need more ...|
|        P2|Large Tapestry Bo...|Polyester cover r...|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|
|        P4|Men's Crazy Horse...|This 9 in. black ...|
|        P5|Mariana 6 ft. Mul...|With robust struc...|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|
|        P9|Traditional Silve...|This transitional...|
|       P10|15 in. x 59 in. O...|Its easy to add a...|
|       P11|1 qt. #350F-7 Wil...|BEHR PREMIUM PLUS...|
|       P12|Anthracite Cordle...|BlindsAvenue ligh...|
|       P13|SlimGrip 78-Inch ...|Luverne SlimGrip ...|
|       P14|6 in. x 28 in. x ...|Our Rustic Collec...|
|       P1

List of 8 products recently viewed by the user.

In [ ]:
recently_viewed_products = [
    'P316',
    'P333',
    'P1115',
    'P1691',
    'P1082',
    'P397',
    'P1441',
    'P1054',
]

### Task 2 - Prepare the dataset

Combine `title` and `description` Columns

In [32]:
df = df.withColumn("combined_text", concat_ws(" ", df.title, df.description))

df.show()

+----------+--------------------+--------------------+--------------------+
|product_id|               title|         description|       combined_text|
+----------+--------------------+--------------------+--------------------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|Men's 3X Large Ca...|
|        P1|Turmode 30 ft. RP...|If you need more ...|Turmode 30 ft. RP...|
|        P2|Large Tapestry Bo...|Polyester cover r...|Large Tapestry Bo...|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|16-Gauge-Sinks Ve...|
|        P4|Men's Crazy Horse...|This 9 in. black ...|Men's Crazy Horse...|
|        P5|Mariana 6 ft. Mul...|With robust struc...|Mariana 6 ft. Mul...|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|5 gal. #650C-2 Po...|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|7/8 in. x 4-1/2 i...|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|Ring Gold Bar Car...|
|        P9|Traditional Silve...|This transitional...|Traditional Silve...|
|       P10|

get the combined_text column and convert it into a list

In [33]:
list_combined_text = df.select("combined_text").rdd.flatMap(lambda x: x).collect()

print(list_combined_text[:4])

["Men's 3X Large Carbon Heather Cotton/Polyester Rain Defender Paxton Heavyweight Hooded Zip-Front Sweatshirt This heavyweight, water-repellent hooded sweatshirt has a zip front for fast layering. ORIGINAL FIT. 13 oz., 75% cotton/25% polyester blend with Rain Defender durable water repellent. Attached, jersey-lined three-piece hood with drawcord closure. Antique-finish brass front zipper. Two front hand-warmer pockets have a hidden security pocket inside. Stretchable, spandex-reinforced rib-knit cuffs and waistband. Locker loop facilitates hanging.", "Turmode 30 ft. RP TNC Female to RP TNC Male Adapter Cable If you need more length between your existing wireless device and Hi-Gain Antenna, this is the product for you. It's compatible with most Wi-Fi Antennas, so it is easy for you to extend your wireless network. Just replace your existing cable that runs between your wireless device and Antenna and you're ready to use your network with extended range.", 'Large Tapestry Bolster Bed Pol

Use OpenAI text embedding model to create the vector embeddings.

In [36]:
# Call the Embedding API
response = openai.Embedding.create(
    input=list_combined_text,
    model="text-embedding-ada-002"  # Use a supported model
)

print(response)

APIRemovedInV1: 

You tried to access openai.Embedding, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


Let't put the embedding vectors into our original dataframe

Convert embedding vectors list into a Pyspark DataFrame

In [14]:
features_column_names = [f"embedding_{i}" for i in range(len(embedding_vectors[0]))]

NameError: name 'embedding_vectors' is not defined

Add unique `row_id` to each row in the pysaprk dataframe

In [38]:
openai migrate

SyntaxError: invalid syntax (<ipython-input-38-424aeda019d3>, line 1)

Add unique `row_id` to each row in our main pyspark dataframe `df`

Let's join the two dataframes

### Task 3 - Cluster products using K-means

Assemble the 512 Embedding Columns into a Single 'features' Column

Apply K-Means Clustering with 5 Clusters on the `features` Column

### Task 4 - Visualize the clusters

Let's reduce the dimensionality of our features for visualization purpose

`512 dimensions => 2 dimensions`

Let's plot the Clusters

In [ ]:
def plot_clusters(pca_df, num_clusters=5):
    """
    Plots a 2D visualization of clusters using Plotly Express.

    Parameters:
    - pca_df (DataFrame): A Pandas DataFrame containing columns 'x', 'y', and 'cluster'.
      'x' and 'y' are the 2D PCA components, and 'cluster' indicates the cluster label.
    - num_clusters (int): The number of unique clusters to display.
    - recently_viewed_df (DataFrame, optional): DataFrame with 'x' and 'y' coordinates for recently viewed products.

    This function creates an interactive scatter plot where each point is colored according to its cluster.
    Recently viewed products are marked as black crosses if provided.

    Returns:
    - fig (Figure): The Plotly figure object for the plot.
    """

    # Create the base cluster plot
    fig = px.scatter(
        pca_df,
        x='x',
        y='y',
        opacity=0.6,
        size_max=4,
        color= pca_df.cluster.astype(str),
        title='2D Visualization of Clusters with Recently Viewed Products',
        labels={'x': 'PCA Component 1', 'y': 'PCA Component 2'},
        category_orders={'cluster': list(range(num_clusters))},
        # show the product id in the tooltip
        hover_data={'product_id': True}

    )

    # Update layout to add legend title and adjust plot settings
    fig.update_layout(legend_title_text='Clusters', legend=dict(x=1, y=1), width=600, height=500)

    return fig

fig = plot_clusters(pca_df)
fig.show()

### Task 5 - Highlight recently viewed products

In [ ]:
print("The user has recently viewed the following products: ", recently_viewed_products)

Let's have a look at the records in our `clustered_data` dataframe related to the recently viewed products.

### Task 6 - Recommend products based on recently viewed products

Let's have a look at the recently viewed products titles

Let's see the distinct clusters of the recenetly viewed products.

Let's find the possible products for the recommendation.

Let's perform a groupby and generate a list of product IDs that can be recommended for each of the clusters.

In [ ]:
# write a python function to display the recommendations
def display_recommendations(row):
  # find the title of the product in df
  product_ids = row['random_recommendations']
  cluster = row.cluster

  titles = data. \
          filter(data["product_id"]. \
          isin(product_ids)).select("title").collect()

  print("\n")
  print("Recommendations for Cluster:", cluster)
  for title in titles:
    print(title[0])

recommendations_df.apply(display_recommendations, axis=1)